## 1. Load dataset

In [1]:
# import library
from typing import List

import numpy as np
import torch
import evaluate

from sklearn.model_selection import train_test_split
import nltk
nltk.download ('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\tienn\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [2]:
#load tree bank dataset
tagged_sentences = nltk.corpus.treebank.tagged_sents()

print("Number of samples:", len(tagged_sentences))

#save sentences and tags
sentences, sentence_tags = [], []

for tagged_sentence in tagged_sentences:
                                      sentence, tags = zip(*tagged_sentence)
                                      sentences.append([word.lower() for word in sentence])
                                      sentence_tags.append([tag for tag in tags])

Number of samples: 3914


## 2. Preprocessing

In [3]:
#Model
from transformers import AutoTokenizer, AutoModelForTokenClassification
model_name = "QCRI/bert-base-multilingual-cased-pos-english"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)

model = AutoModelForTokenClassification.from_pretrained(model_name)

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
len(tokenizer)

119547

In [5]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

### Split dataset into train, validation and test set

In [6]:
train_sentences, test_sentences, train_tags, test_tags = train_test_split(
    sentences,
    sentence_tags,
    test_size=0.3,
    random_state=42
)

valid_sentences, test_sentences, valid_tags, test_tags = train_test_split(
    test_sentences,
    test_tags,
    test_size=0.5,
    random_state=42
)

In [7]:
from collections import defaultdict

label2id = defaultdict(int, model.config.label2id)
id2label = {id: label for label, id in label2id.items()}

In [8]:
from torch.utils.data import Dataset

MAX_LEN = max([len(sentence) for sentence in train_sentences])

class PosTagging_Dataset(Dataset):
    def __init__(self,
                 sentences: List[List[str]],
                 tags: List[List[str]],
                 tokenizer,
                 label2id,
                 max_len=MAX_LEN
                 ):

        super().__init__()
        self.sentences = sentences
        self.tags = tags
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.label2id = label2id

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        input_token = self.sentences[idx]
        label_token = self.tags[idx]

        input_token = self.tokenizer.convert_tokens_to_ids(input_token)
        attention_mask = [1] * len(input_token)
        labels = [self.label2id[token] for token in label_token]

        return {
            "input_ids": self.pad_and_truncate(input_token, pad_id=self.tokenizer.pad_token_id),
            "labels": self.pad_and_truncate(labels, pad_id=label2id["0"]),
            "attention_mask": self.pad_and_truncate(attention_mask, pad_id=0)
        }

    def pad_and_truncate(self, inputs: List[int], pad_id: int):
        if len(inputs) < self.max_len:
            padded_inputs = inputs + [pad_id] * (self.max_len - len(inputs))
        else:
            padded_inputs = inputs[:self.max_len]
        return torch.as_tensor(padded_inputs)

### Dataset loader

In [9]:
train_dataset = PosTagging_Dataset(train_sentences, train_tags, tokenizer, label2id)
val_dataset = PosTagging_Dataset(valid_sentences, valid_tags, tokenizer, label2id)
test_dataset = PosTagging_Dataset(test_sentences, test_tags, tokenizer, label2id)

## 3. Metric

In [10]:
accuracy = evaluate.load("accuracy")

ignore_label = len(label2id)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mask = labels != ignore_label
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions[mask], references=labels[mask])

## 4. Trainer

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\tienn\AppData\Local\Temp\ipykernel_34840\3587467276.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## 5. Training

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.044969,0.987302
2,No log,0.037109,0.989439
3,0.148700,0.033439,0.990376
4,0.148700,0.031439,0.990941
5,0.148700,0.030208,0.991331
6,0.029700,0.029267,0.991677
7,0.029700,0.028885,0.991734
8,0.029700,0.028618,0.991928
9,0.023800,0.028505,0.991922
10,0.023800,0.028444,0.991941


TrainOutput(global_step=1720, training_loss=0.06159098231515219, metrics={'train_runtime': 359.6619, 'train_samples_per_second': 76.155, 'train_steps_per_second': 4.782, 'total_flos': 3789641345256360.0, 'train_loss': 0.06159098231515219, 'epoch': 10.0})

## 6. Inference

In [13]:
# tokenization
test_sentence = "We are exploring the topic of deep learning"
input = torch.as_tensor([tokenizer.convert_tokens_to_ids(test_sentence.split())])
input = input.to("cuda")

# prediction
outputs = model(input)
_, preds = torch.max(outputs.logits, -1)
preds = preds[0].cpu().numpy()

# decode
pred_tags = ""
for pred in preds:
    pred_tags += id2label[pred] + " "
pred_tags # => PRP VBP VBG DT NN IN JJ NN

'PRP VBP RB DT NN IN JJ NN '